In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 22
TITLE = "15-16 February 2021"
SUBTITLE = "8 AM 16 February 2021"
sts = utc(2021, 2, 16, 0)
ets = utc(2021, 2, 17, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [11]:
cull = ['IA-WY-3']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['IA-CN-17', 1],
    [1000, 2.5],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=1, upper=5)
res = mp.postprocess(filename='210216.png')
mp.close()

     state  wfo  val        lon       lat                              geo  \
596     VT  BTV  4.9  -73.06000  44.85000   POINT (2074761.441 334491.101)   
597     MS  MEG  4.9  -89.67000  34.95000  POINT (942241.123 -1058502.768)   
598     NY  BUF  4.9  -78.05000  43.00000    POINT (1758519.836 16648.392)   
599     MO  SGF  4.9  -93.95000  37.39000   POINT (535103.582 -826240.958)   
600     CO  GJT  4.9 -106.95000  39.17000  POINT (-599047.442 -622919.325)   
...    ...  ...  ...        ...       ...                              ...   
1609    IA  DVN  1.0  -91.26944  41.58222   POINT (724784.868 -341444.962)   
1610    NY  BGM  1.0  -75.51970  42.51170    POINT (1969740.443 22194.395)   
1611    NH  GYX  1.0  -71.18330  44.45000   POINT (2227291.308 342482.996)   
1612    NH  BOX  1.0  -72.31310  42.99630   POINT (2198224.968 156878.209)   
1646    IA  DVN  1.0  -90.49841  41.97619   POINT (783539.197 -290755.957)   

      used_for_analysis     nwsli  plotme source  
596         